In [1]:
from utils.schedule_utils import load_data_split,table_to_str,table_to_str_sql,find_intersection_and_add_row_id,Prepare_Data_for_Operator_Sequence,format_document,batch_rerank_scores,ROLLBACK,merge_clean_and_format_df_dict,retrieve_rows_by_subtables,process_error_analysis_list
import pickle
import numpy as np
import pandas as pd
import os
import json
from FlagEmbedding import FlagReranker
from tqdm import tqdm
from typing import Dict, Any
from utils.evaluator import Evaluator
from utils.prompt_generate import build_wikitq_prompt_from_df,evaluate_predictions,filter_dataframe_from_responses,fix_sql_query,match_subtables,retrieve_rows_by_subtables,build_tab_fact_prompt_from_df
from utils.async_llm import infer_prompts

/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
ALL_LABELS = ['Base', 'Execute_SQL', 'RAG', 'Select_Column', 'Select_Row']
def parse_valid_route(input):
    try:
        parse = eval(input)[0]
        eval_parse = eval(parse)
        result = [e for e in eval_parse if e in ALL_LABELS]
    except:
        result = []
        for label in ALL_LABELS:
            if input.lower().__contains__(label.lower()):
                result.append(label)
    ## 如果有非法字符
    return result

In [3]:
os.environ["VLLM_ATTENTION_BACKEND"] = "FLASHINFER"

In [4]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from time import sleep
import argparse
from vllm.lora.request import LoRARequest
import json
from tqdm import tqdm
import os 

INFO 10-20 19:30:47 [__init__.py:216] Automatically detected platform cuda.


In [5]:
## 参数记录 for tab_fact
N_PARALLEL = 32 ## only for convert_df_type_parallel.py of data preprocess，一次性离线的
embedding_base_model_path = '/data/workspace/yanmy/models/bge-m3'
router_model_path = '/data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-router-tab_fact-hn' ## fine-tuned router model path
check_moedl_path = '/data/workspace/yanmy/HybridRAG/H-STAR/check/output/bge-reranker-v2-m3-finetuned-tab_fact' ## check model path
tmp_save_path = 'datasets/schedule_test/tab_fact' ## 临时存储路径
dataset_name = 'tab_fact'
split = 'test_small'
tau = 0.75
check_tau = 0.8
ALL_LABELS = [
    'Base', 'Select_Row', 'Select_Column', 'Execute_SQL', 'RAG_20_5', 
]


In [6]:
RAG_20_5 = np.load('../datasets/ablation/tab_fact_test_small_RAG_Hybrid.npy',allow_pickle=True).item()
wikitq_df_processed = np.load('datasets/schedule_test/tab_fact/wikitq_df_processed.npy',allow_pickle=True).item()
rewrite_query_list = np.load('datasets/schedule_test/tab_fact/rewrite_query.npy',allow_pickle=True)
training_free_router = pd.read_csv('../datasets/ablation/tab_fact_test_small_training_free_router_output_4B.csv',index_col=0)
training_free_router = [parse_valid_route(x) for x in training_free_router['predict'].tolist()]

In [7]:
processed_table = np.load('../datasets/check/tab_fact_test_small_processed_table.npy',allow_pickle=True).item()

In [8]:
### 读取必要文件 （step 2）
dataset = load_data_split(dataset_name,split)
wikitq_df_processed = np.load('datasets/schedule_test/tab_fact/wikitq_df_processed.npy',allow_pickle=True).item()


In [9]:
## step 5
# with open(f'{tmp_save_path}/inference_result.pkl', 'rb') as f:
#     error_analysis_row = pickle.load(f)
# ranked_result = process_error_analysis_list(error_analysis_row, truncate=True, tau=tau)
ranked_result = process_error_analysis_list(training_free_router, truncate=True, tau=tau)

- Run Check Model

In [10]:
def Prepare_Data_for_Operator_Sequence(index,sequence,dataset, processed_table, remove_operator = []):
    data_entry = {
    'id': dataset[index]['id'],
    'query': dataset[index]['question'], ## 没有ReWrite，暂时不做变化
    # 'table': wikitq_df_processed[index],
    'title': dataset[index]['table']['page_title'], ## 不变
    # 'SQL': sql_command['sql'] + table_to_str(sql_command['table']) ,
        }
    sequence = [s for s in sequence if s not in remove_operator]
    if sequence.__contains__('Execute_SQL'): ## 有Execute_SQL计算符
        sql_list = [sql_count for sql_count in processed_table['Execute_SQL_count'] if sql_count['id']==index]
        # sql_command = sql_list[0]
        if len(sql_list)==0:
            SQL = ''
        else:
            sql_command = sql_list[0]
            SQL = sql_command['sql'] + table_to_str(sql_command['table'])
        data_entry['SQL'] = SQL
    else:
        data_entry['SQL'] = ''
    extraction_sequence = [s for s in sequence if s!='Execute_SQL']
    # print(extraction_sequence)
    if len(extraction_sequence)==1: ## Only One Operator
        method = extraction_sequence[0]
        data_entry['table'] = processed_table[method][index]
    elif len(extraction_sequence)==2:
        table_1 = processed_table[extraction_sequence[0]][index]
        table_2 = processed_table[extraction_sequence[1]][index]
        # print(table_1.shape, table_2.shape)
        table_intersection = find_intersection_and_add_row_id(table_1,table_2)
        data_entry['table'] = table_intersection
    elif len(extraction_sequence)==3:
        table_1 = processed_table[extraction_sequence[0]][index]
        table_2 = processed_table[extraction_sequence[1]][index]
        table_3 = processed_table[extraction_sequence[2]][index]
        # print(table_1.shape, table_2.shape)
        table_intersection = find_intersection_and_add_row_id(table_1,table_2)
        table_intersection = find_intersection_and_add_row_id(table_intersection,table_3)
        data_entry['table'] = table_intersection
    elif len(extraction_sequence)==0:
        data_entry['table'] = processed_table['Base'][index]
    assert data_entry.__contains__('SQL')
    assert data_entry.__contains__('table')
    return data_entry

In [11]:
# 对每一条数据，维护一个 Sequence_Operator
# 每条数据应该对应一个 Dict,这个Dict 包含了3 个参数：
    # id: index
    # Sequence: Operator Sequence
    # Terminated: True/False
    # Check Score: CrossEncoder 分数
    # Check Status: True/False,当前是否被计算
# 对每条数据，如果 Terminated = True，则不参与计算
# 对应的，如果 Terminated = False，那么查看 Check Status,如果 Statue=False，那么计算 Check_Score
# 每一轮计算之后，Check_Score 更新，Check Statue更新，该轮开始判定，用一个 threshold
# 如果 Check Status = True, Check Score >= Threshold，那么标注 Terminated = True，当前路径被接收
# 如果 Check Status = True, Check Score < Threshold，那么Terminated = False, Check Status = False, Check Score 清空
# Check Score 触发 RollBack 机制，即除了‘Execute_SQL'外，Extract 回退，重新计算当前流程
# 如果当前操作符只有'Execute_SQL’和空（所有 RollBack 均未被接收），那么触发 FORWARD,Terminated  = True，强制修改 Operator Sequence 为['Execute_SQL']
Check_Model_Data_Sequence = {}
for key in ranked_result.keys(): ## 遍历所有数据,初始化
    start_sequence = ranked_result[key]
    Check_Model_Data_Sequence[key] = {}
    Check_Model_Data_Sequence[key]['id'] = key
    Check_Model_Data_Sequence[key]['Sequence'] = start_sequence
    if start_sequence == ['Base'] or start_sequence == ['Execute_SQL']: ## Terminated 不参与计算
        Check_Model_Data_Sequence[key]['Terminated'] = True
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
    else:
        Check_Model_Data_Sequence[key]['Terminated'] = False
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
# Check_Model_Data_Sequence[4]
for key in Check_Model_Data_Sequence.keys():
    data_entry = Prepare_Data_for_Operator_Sequence(key,Check_Model_Data_Sequence[key]['Sequence'],dataset, processed_table)
    Check_Model_Data_Sequence[key]['data_entry'] = data_entry
## Check
    

- load ReRanking Model

In [16]:
reranker_model = FlagReranker(check_moedl_path, use_fp16=True, devices='cuda:0')

In [17]:
check_tau = check_tau
for loop in range(3):
    updated_data = batch_rerank_scores(reranker_model, Check_Model_Data_Sequence, batch_size=16)
    Check_Model_Data_Sequence = updated_data
    ## Check Terminal Status
    for key in Check_Model_Data_Sequence.keys():
        if Check_Model_Data_Sequence[key]['Terminated'] == True:
            continue
        else:
            if Check_Model_Data_Sequence[key]['Check_Status'] == True:
                if Check_Model_Data_Sequence[key]['Check_Score'] >= check_tau: ## 0.5 threshold
                    Check_Model_Data_Sequence[key]['Terminated'] = True
                else:
                    Check_Model_Data_Sequence[key]['Terminated'] = False
                    Check_Model_Data_Sequence[key]['Check_Status'] = False
                    Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
                    current_sequence = Check_Model_Data_Sequence[key]['Sequence']
                    ROLLBACK_seq, terminated_flag = ROLLBACK(current_sequence)
                    Check_Model_Data_Sequence[key]['Sequence'] = ROLLBACK_seq
                    Check_Model_Data_Sequence[key]['Terminated'] = terminated_flag
                    if terminated_flag == False:
                        data_entry = Prepare_Data_for_Operator_Sequence(key,ROLLBACK_seq,dataset, processed_table)
                        Check_Model_Data_Sequence[key]['data_entry'] = data_entry
            else:
                raise ValueError("Check Status should be True when Terminated is False after reranking.")

Filtering items to rerank (Terminated=False and data_entry exists)...


Processed prompts:   0%|          | 0/2024 [8:12:46<?, ?it/s]
Task exception was never retrieved
future: <Task finished name='Task-2025' coro=<run_inference.<locals>.wrapped_task() done, defined at /data/workspace/yanmy/HybridRAG/H-STAR/utils/async_llm.py:120> exception=NotFoundError("Error code: 404 - {'error': {'message': 'The model `/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}}")>
Traceback (most recent call last):
  File "/home/aizoo/miniconda3/envs/hstar/lib/python3.9/asyncio/tasks.py", line 256, in __step
    result = coro.send(None)
  File "/data/workspace/yanmy/HybridRAG/H-STAR/utils/async_llm.py", line 121, in wrapped_task
    res = await infer_one(client, model, p, sem, max_retries, request_timeout, temperature, top_p, max_tokens, n, presence_penalty)
  File "/data/workspace/yanmy/HybridRAG/H-STAR/utils/async_llm.py", line 61, in infer_one
    resp = await client.chat.completions.create(
  File "/h

Computing scores for 2024 items with batch size 16...


Compute Scores: 100%|██████████| 127/127 [00:03<00:00, 36.57it/s]

Updating scores in the original dictionary...
Reranking complete.
Filtering items to rerank (Terminated=False and data_entry exists)...
No items to process.
Filtering items to rerank (Terminated=False and data_entry exists)...
No items to process.


In [18]:
reranker_model = FlagReranker(check_moedl_path, use_fp16=True)

In [ ]:
## step 10
SQL_list_final = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    if sequence==[] or sequence.__contains__('Execute_SQL'):
        SQL_list_final.append(index)
add_sql_list = list(set(SQL_list_final) - set(LLM_query_list['Execute_SQL']['index']))
add_sql_query_list = []
for index in add_sql_list:
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/sql_reason_tabfact.txt',processed=True)
    add_sql_query_list.extend([prompt])
add_sql_response_list = response_vllm(add_sql_query_list, sample_num=3)

In [ ]:
## step 11: generate QA result
prompt_list = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    prompt = build_wikitq_prompt_from_df(dataset,Check_Model_Data_Sequence[index]['data_entry']['table'],index,template_path='../prompts/text_reason_tabfact.txt',processed=True)
    if sequence==[] or sequence.__contains__('Execute_SQL'):
    # if sequence.__contains__('Execute_SQL'):
        # try:
        evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
        prompt = prompt + evidence
        print(f'SQL at index {index}')
    prompt_list.append(prompt)
# wikitq_df['instruction'] = prompt_list

: 

In [14]:
## step 11: execute QA
# qa_final = response_vllm(llm,prompt_list, sample_num=1)
qa_final = infer_prompts(prompt_list,temperature=0, top_p=1, sample_num=1,llm_path = '/data/workspace/yanmy/models/Qwen3-30B-A3B-Instruct-2507-FP8')
wikitq_df = pd.DataFrame(dataset)
wikitq_df['instruction'] = prompt_list
wikitq_df['predict'] = qa_final[0]
## step 12: evaluate
wikitq_df['predict'] = [str(s) for s in qa_final[0]]
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df, dataset)

Processed prompts: 100%|██████████| 2024/2024 [02:33<00:00, 13.21it/s, est. speed input=28679.35 toks/s, output=2426.97 toks/s]


  0%|          | 0/2024 [00:00<?, ?it/s]

Correct Samples: 1883; Total Samples: 2024
Accuracy: 93.03


Correct Samples: 1750; Total Samples: 2024
Accuracy: 86.46（Training-Free Router）
Correct Samples: 1765; Total Samples: 2024
Accuracy: 87.20 (w/o Check)
Correct Samples: 1763; Total Samples: 2024
Accuracy: 87.10 (w/o table extraction)
Correct Samples: 1775; Total Samples: 2024
Accuracy: 87.70 （w/o SQL）
Correct Samples: 1773; Total Samples: 2024
Accuracy: 87.55 （w/o RAG）